In [1]:
import sqlite3

conn = sqlite3.connect('ODM_project.db')
cursor = conn.cursor()

# Enable Foreign Key support
cursor.execute("PRAGMA foreign_keys = ON;")

# Create the Movie table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Movie (
    movie_id TEXT PRIMARY KEY,
    release_date DATE,
    title TEXT,
    production_company TEXT,
    duration INTEGER,
    rating TEXT,
    genres TEXT,
    tagline TEXT,
    website TEXT,
    awards TEXT,
    overall_meta_score INTEGER,
    overall_user_score INTEGER
)''')

# Create the Sales table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Sales (
    movie_id TEXT PRIMARY KEY,
    budget FLOAT,
    gross_domestic FLOAT,
    gross_international FLOAT,
    gross_worldwide FLOAT,
    opening_weekend FLOAT,
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id)
)''')

# Create the User table
cursor.execute('''
CREATE TABLE IF NOT EXISTS User (
    user_id INT PRIMARY KEY,
    reviewer TEXT,
    total_reviews INT
)''')

# Create the UserReview table
cursor.execute('''
CREATE TABLE IF NOT EXISTS UserReview (
    review_id TEXT PRIMARY KEY,
    movie_id TEXT,
    user_id INT,
    user_score INT,
    DateP DATE,
    summary TEXT,
    FOREIGN KEY (user_id) REFERENCES User (user_id),
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id)
)''')

# Create the Expert table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Expert (
    expert_id INT PRIMARY KEY,
    reviewer TEXT,
    total_reviews INT
)''')

# Create the ExpertReview table
cursor.execute('''
CREATE TABLE IF NOT EXISTS ExpertReview (
    review_id TEXT PRIMARY KEY,
    movie_id TEXT,
    expert_id INT,
    meta_score INT,
    DateP DATE,
    summary TEXT,
    FOREIGN KEY (expert_id) REFERENCES Expert (expert_id),
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id)
)''')

# Create the Film_crew table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Film_Crew (
    person_id INTEGER PRIMARY KEY,
    FullName TEXT
)''')

# Create Junction Tables (Cast, Director, Writer)
cursor.execute('''
CREATE TABLE IF NOT EXISTS MovieCast (
    movie_id TEXT,
    person_id INTEGER,
    CharacterName TEXT,
    PRIMARY KEY (movie_id, person_id),
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id),
    FOREIGN KEY (person_id) REFERENCES Film_crew (person_id)
)''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Director (
    movie_id TEXT,
    person_id INTEGER,
    PRIMARY KEY (movie_id, person_id),
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id),
    FOREIGN KEY (person_id) REFERENCES Film_crew (person_id)
)''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Writer (
    movie_id TEXT,
    person_id INTEGER,
    PRIMARY KEY (movie_id, person_id),
    FOREIGN KEY (movie_id) REFERENCES Movie (movie_id),
    FOREIGN KEY (person_id) REFERENCES Film_crew (person_id)
)''')

conn.commit()
conn.close()

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('ODM_project.db')

# 1. Get a list of all table names in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

print("--- DATABASE SCHEMA VERIFICATION ---")

for table_name in tables['name']:
    # 2. Query 0 rows to just get the column headers
    df_schema = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 0;", conn)
    
    print(f"\nTable: {table_name}")
    print(f"Columns: {list(df_schema.columns)}")

conn.close()

--- DATABASE SCHEMA VERIFICATION ---

Table: Movie
Columns: ['movie_id', 'release_date', 'title', 'production_company', 'duration', 'rating', 'genres', 'tagline', 'website', 'awards', 'overall_meta_score', 'overall_user_score']

Table: Sales
Columns: ['movie_id', 'budget', 'gross_domestic', 'gross_international', 'gross_worldwide', 'opening_weekend']

Table: User
Columns: ['user_id', 'reviewer', 'total_reviews']

Table: UserReview
Columns: ['review_id', 'movie_id', 'user_id', 'user_score', 'DateP', 'summary']

Table: Expert
Columns: ['expert_id', 'reviewer', 'total_reviews']

Table: ExpertReview
Columns: ['review_id', 'movie_id', 'expert_id', 'meta_score', 'DateP', 'summary']

Table: Film_Crew
Columns: ['person_id', 'FullName']

Table: MovieCast
Columns: ['movie_id', 'person_id', 'CharacterName']

Table: Director
Columns: ['movie_id', 'person_id']

Table: Writer
Columns: ['movie_id', 'person_id']


In [5]:
import pandas as pd
import sqlite3
import json
import re

# --- STEP 1: ROBUST REGEX LOAD ---
def robust_load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    raw_objects = re.findall(r'\{.*?\}', content, re.DOTALL)
    data = []
    for obj in raw_objects:
        try:
            data.append(json.loads(obj))
        except json.JSONDecodeError:
            continue
    return pd.DataFrame(data)

df = robust_load_json('chrisOutput_22_1_26.json')

# --- STEP 2: DYNAMIC ID GENERATION & CLEANING ---

# 1. Team Standard ID Function (Share this with your teammates!)
def create_team_movie_id(title, release_date):
    if pd.isna(title): return None
    # Clean title: lowercase, remove special characters, replace spaces with dashes
    clean_title = re.sub(r'[^a-z0-9\s]', '', str(title).lower().strip())
    clean_title = clean_title.replace(" ", "-")
    
    # Extract Year (handles different date formats)
    year_match = re.search(r'\d{4}', str(release_date))
    year = year_match.group(0) if year_match else "0000"
    
    return f"{clean_title}-{year}"

# 2. Apply the new ID logic immediately
# This replaces the IMDb 'tt...' IDs with your team-standard slugs
df['movie_id'] = df.apply(lambda x: create_team_movie_id(x['title'], x['release_date']), axis=1)

# Reviewer and Crew Lookups
unique_users = df[df['reviewer_name'].notnull()]['reviewer_name'].unique()
user_lookup = pd.DataFrame({'reviewer': unique_users, 'user_id': range(1, len(unique_users) + 1)})

all_crew = pd.concat([df.get('stars', pd.Series()).explode(), df.get('director'), df.get('writer', pd.Series()).explode()]).unique()
filtered_crew = [name for name in all_crew if name and str(name).lower() != 'none']
crew_lookup = pd.DataFrame({'FullName': filtered_crew, 'person_id': range(1, len(filtered_crew) + 1)})




# --- STEP 3: MAPPING TO SQL TABLES ---
conn = sqlite3.connect('ODM_project.db')
cursor = conn.cursor()

try:
    # A. CLEAR PREVIOUS DATA (To avoid IntegrityErrors)
    # Correcting 'Filmcrew' to 'Film_crew' based on your verified schema
    cursor.execute("DELETE FROM Movie;")
    cursor.execute("DELETE FROM Sales;")
    cursor.execute("DELETE FROM User;")
    cursor.execute("DELETE FROM UserReview;")
    cursor.execute("DELETE FROM Film_crew;")
    cursor.execute("DELETE FROM MovieCast;")
    cursor.execute("DELETE FROM Director;")
    cursor.execute("DELETE FROM Writer;")
    conn.commit()

    # B. MOVIE TABLE (Drop duplicates to satisfy UNIQUE constraint)
    df_movie = df[df['title'].notnull()].copy()
    df_movie = df_movie.rename(columns={'dynamic_movie_id': 'movie_id'})
    df_movie = df_movie.drop_duplicates(subset=['movie_id'])

    movie_cols = ['movie_id', 'release_date', 'title', 'production_company', 'duration', 
                  'rating', 'genres', 'tagline', 'website', 'awards']
    
    for col in movie_cols:
        if col not in df_movie.columns: df_movie[col] = None

    df_movie['genres'] = df_movie['genres'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df_movie['production_company'] = df_movie['production_company'].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
    df_movie[movie_cols].to_sql('Movie', conn, if_exists='append', index=False)

    # C. SALES TABLE
    df_sales = df[df['title'].notnull()].copy()
    df_sales = df_sales.rename(columns={'dynamic_movie_id': 'movie_id', 'grossworldwide': 'gross_worldwide', 'openingweekend': 'opening_weekend'})
    df_sales = df_sales.drop_duplicates(subset=['movie_id'])

    def clean_money(val):
        if pd.isna(val) or val is None: return None
        num = re.sub(r'[^\d.]', '', str(val))
        return float(num) if num else None

    df_sales['budget'] = df_sales['budget'].apply(clean_money)
    df_sales['gross_worldwide'] = df_sales['gross_worldwide'].apply(clean_money)
    df_sales['opening_weekend'] = df_sales['opening_weekend'].apply(clean_money)
    df_sales[['movie_id', 'budget', 'gross_worldwide', 'opening_weekend']].to_sql('Sales', conn, if_exists='append', index=False)

    # D. USER & REVIEWS
    df_counts = df.groupby('reviewer_name').size().reset_index(name='total_reviews')
    df_user_final = user_lookup.merge(df_counts, left_on='reviewer', right_on='reviewer_name')[['user_id', 'reviewer', 'total_reviews']]
    df_user_final.to_sql('User', conn, if_exists='append', index=False)

    df_rev = df[df['reviewer_name'].notnull()].copy()
    df_rev = df_rev.merge(user_lookup, left_on='reviewer_name', right_on='reviewer')
    df_rev = df_rev.rename(columns={'dynamic_movie_id': 'movie_id', 'review_date': 'DateP', 'review_score': 'user_score', 'review_text': 'summary'})
    df_rev['review_id'] = range(1, len(df_rev) + 1)
    df_rev[['review_id', 'movie_id', 'user_id', 'user_score', 'DateP', 'summary']].to_sql('UserReview', conn, if_exists='append', index=False)

    # E. FILM_CREW 
    # Use 'Film_Crew' with a Capital C to match your database exactly
    crew_lookup.to_sql('Film_Crew', conn, if_exists='append', index=False)

# --- STEP 4: POPULATE JUNCTION TABLES WITH CHARACTER MAPPING ---

    def load_junction(df, source_col, table_name, person_df, role_col=None):
        junction_rows = []
        
        for _, row in df.iterrows():
            movie_id = row['movie_id']
            people = row.get(source_col)
            roles = row.get(role_col) if role_col else None
            
            # If it's a list (like stars or writers), iterate through it
            if isinstance(people, list):
                for i, person_name in enumerate(people):
                    # Map the role to the person by index (if roles list exists)
                    char_mapping = roles[i] if roles and i < len(roles) else None
                    junction_rows.append({
                        'movie_id': movie_id,
                        'FullName': person_name,
                        'CharacterName': char_mapping
                    })
            # If it's a single string (like director), just add it
            elif pd.notnull(people):
                junction_rows.append({
                    'movie_id': movie_id,
                    'FullName': people,
                    'CharacterName': None
                })

        # Convert collected rows to a DataFrame and merge with numeric person_id
        if junction_rows:
            df_temp = pd.DataFrame(junction_rows)
            final_junction = df_temp.merge(person_df, on='FullName')
            
            # Select columns based on the table requirements
            cols = ['movie_id', 'person_id']
            if table_name == 'MovieCast':
                cols.append('CharacterName')
                
            final_junction[cols].to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Successfully loaded {len(final_junction)} rows into {table_name}.")

    # --- WRAP THE EXECUTION IN A PROPER TRY/FINALLY BLOCK ---
    try:
        # ... (Your existing code to clear data and load Movie/Sales/User tables) ...

        # 1. Load Directors and Writers (No character names needed)
        load_junction(df, 'director', 'Director', crew_lookup)
        load_junction(df, 'writer', 'Writer', crew_lookup)
        
        # 2. Load MovieCast (Mapping 'stars' to 'roles' for the CharacterName column)
        load_junction(df, 'stars', 'MovieCast', crew_lookup, role_col='roles')

        print("Success! All tables populated.")

    except Exception as e:
        print(f"Error during population: {e}")

finally:
    # FIXED: This must be at the same indentation level as the 'try' above
    conn.close()
    print("Database connection closed.")


Successfully loaded 93 rows into Director.
Successfully loaded 150 rows into Writer.
Successfully loaded 1240 rows into MovieCast.
Success! All tables populated.
Database connection closed.


In [7]:
import sqlite3
import pandas as pd

# Connect to your database
conn = sqlite3.connect('ODM_project.db')

# 1. Get a list of all tables you created
tables_query = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
tables = pd.read_sql_query(tables_query, conn)['name'].tolist()

print("--- DATABASE VERIFICATION REPORT ---")

for table in tables:
    print(f"\n{'='*40}")
    print(f"TABLE: {table}")
    print(f"{'='*40}")
    
    # Check Total Record Count
    count_df = pd.read_sql_query(f"SELECT COUNT(*) as total FROM {table}", conn)
    print(f"Total Rows: {count_df['total'][0]}")
    
    # Check Schema (Column names and Types)
    schema_df = pd.read_sql_query(f"PRAGMA table_info({table})", conn)
    print("\nSchema:")
    print(schema_df[['name', 'type', 'pk']])
    
    # Preview the Data
    print("\nData Preview (First 3 rows):")
    sample_df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 3", conn)
    display(sample_df) # Use display() for a nice table format in Jupyter/VS Code

conn.close()

--- DATABASE VERIFICATION REPORT ---

TABLE: Movie
Total Rows: 93

Schema:
                  name     type  pk
0             movie_id     TEXT   1
1         release_date     DATE   0
2                title     TEXT   0
3   production_company     TEXT   0
4             duration  INTEGER   0
5               rating     TEXT   0
6               genres     TEXT   0
7              tagline     TEXT   0
8              website     TEXT   0
9               awards     TEXT   0
10  overall_meta_score  INTEGER   0
11  overall_user_score  INTEGER   0

Data Preview (First 3 rows):


,movie_id,release_date,title,production_company,duration,rating,genres,tagline,website,awards,overall_meta_score,overall_user_score
0,polite-society-2023,"April 28, 2023",Polite Society,"Focus Features, Parkville Pictures, Reliance F...",1h 44m,6.6,"Action, Comedy",None,None,None,None,None
1,stephen-curry-underrated-2023,"July 21, 2023",Stephen Curry: Underrated,"A24, Apple Original Films, Proximity Media",1h 50m,7.3,"Documentary, Biography, Sport",None,None,None,None,None
2,little-dixie-2023,"February 3, 2023",Little Dixie,"Roxwell Films, Three Point Capital (TPC)",1h 45m,4.9,"Action, Crime, Thriller",None,None,None,None,None



TABLE: Sales
Total Rows: 93

Schema:
                  name   type  pk
0             movie_id   TEXT   1
1               budget  FLOAT   0
2       gross_domestic  FLOAT   0
3  gross_international  FLOAT   0
4      gross_worldwide  FLOAT   0
5      opening_weekend  FLOAT   0

Data Preview (First 3 rows):


,movie_id,budget,gross_domestic,gross_international,gross_worldwide,opening_weekend
0,polite-society-2023,None,None,None,2680713.0,817740.0
1,stephen-curry-underrated-2023,None,None,None,NaN,NaN
2,little-dixie-2023,None,None,None,NaN,NaN



TABLE: User
Total Rows: 587

Schema:
            name  type  pk
0        user_id   INT   1
1       reviewer  TEXT   0
2  total_reviews   INT   0

Data Preview (First 3 rows):


...

,movie_id,person_id,CharacterName
0,polite-society-2023,1,Clara
1,polite-society-2023,2,Ria
2,polite-society-2023,3,Salim Shah



TABLE: Director
Total Rows: 93

Schema:
        name     type  pk
0   movie_id     TEXT   1
1  person_id  INTEGER   2

Data Preview (First 3 rows):


,movie_id,person_id
0,polite-society-2023,1239
1,stephen-curry-underrated-2023,1240
2,little-dixie-2023,1241



TABLE: Writer
Total Rows: 150

Schema:
        name     type  pk
0   movie_id     TEXT   1
1  person_id  INTEGER   2

Data Preview (First 3 rows):


,movie_id,person_id
0,polite-society-2023,1239
1,little-dixie-2023,1241
2,the-disappearance-of-shere-hite-2024,60
